In [103]:
# Importamos las librerias
import pandas as pd
import json
import re
import os

# Extract


In [104]:
movies_df = pd.read_csv("../../data/data_bruta/movies_dataset.csv")
credits_df = pd.read_csv("../../data/data_bruta/credits.csv")

C:\Users\sramr\AppData\Local\Temp\ipykernel_4288\3550380012.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_df = pd.read_csv("../../data/data_bruta/movies_dataset.csv")


In [105]:
movies_df.shape, credits_df.shape

((45466, 24), (45476, 3))

- La data se encontraba fragmentada, por tanto toca unirlos para no perder congruencia.

In [106]:
def merge_movies_and_credits(movies_df: pd.DataFrame, credits_df: pd.DataFrame):
    '''
    La función devuelve el DataFrame resultado de unir movies y credits. Para ello elimina los id's que no están en
    el formato adecuado (número entero), elimina los id's duplicados y finalmente los convierte a enteros de 64 bits.
    '''
    
    # Convierte los id's de cada DataFrame a entero y si no se puede lo etiqueta con NaN
    movies_df['id'] = pd.to_numeric(movies_df['id'], errors='coerce')
    credits_df['id'] = pd.to_numeric(credits_df['id'], errors='coerce')
    
    # Dropea los registros que son NaN
    movies_df = movies_df.dropna(subset=['id'])
    credits_df = credits_df.dropna(subset=['id'])
    
    # Dropea los registros duplicados
    movies_df.drop_duplicates(subset='id', inplace=True)
    credits_df.drop_duplicates(subset='id', inplace=True) 

    # Juntar los DataFrames utilizando el campo "id"
    merged_data = pd.merge(movies_df, credits_df, on='id')
    
    # Eliminar los registros que no tienen título 
    merged_data = merged_data.dropna(subset=['title'])
    
    # Restablecer el índice
    merged_data = merged_data.reset_index(drop=True)
    
    return merged_data


df = merge_movies_and_credits(movies_df, credits_df)

C:\Users\sramr\AppData\Local\Temp\ipykernel_4288\2528066862.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df.drop_duplicates(subset='id', inplace=True)


In [107]:
df.shape    # Como se observa, se ha perdido casi 40 registros (nada polemico)

(45429, 26)

# Transform

- **Transformacion 1:** 
      - 'Desanidar los campos json'

In [108]:
colecciones_nulos = df['belongs_to_collection'].isnull().sum()
generos_nulos = len(df[df['genres'] == '[]'])
productoras_nulos = len(df[df['production_companies'] == '[]'])
paises_nulos = len(df[df['production_countries'] == '[]'])
lenguajes_nulos = len(df[df['spoken_languages'] == '[]'])
actores_nulos = len(df[df['cast'] == '[]'])
directores_nulos = len(df[df['crew'] == '[]'])

In [109]:
colecciones_nulos, generos_nulos, productoras_nulos, paises_nulos, lenguajes_nulos, actores_nulos, directores_nulos

(40942, 2441, 11867, 6278, 3826, 2414, 771)

In [110]:
def convert_column_json(df: pd.DataFrame, column: str, name='name', num_segmentos=507):
    '''
    Esta funcion transforma la columna especificada en un formato mas simple
    Requiere el dataframe y el nombre de la columna 
    Retorna el dataframe con la columna modificada
    '''
    # Calcula el tamaño de cada segmento
    tam_segmento = len(df) // num_segmentos

    for i in range(num_segmentos):
        # Calcula los índices inicial y final para el segmento actual
        start_idx = i * tam_segmento
        end_idx = (i + 1) * tam_segmento

        # Si es el último segmento, ajusta el índice final al tamaño del DataFrame
        if i == num_segmentos - 1:
            end_idx = len(df)

        # Obtiene el segmento actual del DataFrame
        segmento_df = df.iloc[start_idx:end_idx]

        # Aplica la transformación a la columna especificada usando el método apply
        segmento_df[column] = segmento_df[column].apply(lambda x: extract_values_of_dictionaries(x, name))

        # Hace que los cambios se vean reflejados en el dataframe original
        df.iloc[start_idx:end_idx] = segmento_df

    return df

def extract_values_of_dictionaries(value, name):
    # Transforma la fila en la columna especificada en una cadena de texto
    fila = str(value)
    # Utiliza expresión regular para encontrar los substrings entre {} (los diccionarios en la fila)
    diccionarios_en_la_fila = re.findall(r'{.*?}', fila)

    # Una lista de los valores de los diccioarios asociados al film actual en el loop
    valores_row = []

    # Iterar sobre los diccionarios_en_la_fila
    for diccionario in diccionarios_en_la_fila:
        # Buscar la expresión regular 'name' dentro del diccionario
        match = re.search(rf"{name}': '([^']*)", diccionario)
        if match:
            # Le asigna el valor encontrado a la variable valor
            valor = match.group(1)
            valores_row.append(valor)

    # Devuelve la lista de géneros como string
    return json.dumps(valores_row)

In [111]:
df = convert_column_json(df, 'genres')

C:\Users\sramr\AppData\Local\Temp\ipykernel_4288\1949036155.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  segmento_df[column] = segmento_df[column].apply(lambda x: extract_values_of_dictionaries(x, name))
C:\Users\sramr\AppData\Local\Temp\ipykernel_4288\1949036155.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  segmento_df[column] = segmento_df[column].apply(lambda x: extract_values_of_dictionaries(x, name))
C:\Users\sramr\AppData\Local\Temp\ipykernel_4288\1949036155.py:23: SettingWithCopyWarnin

In [112]:
df = convert_column_json(df, 'production_companies')

C:\Users\sramr\AppData\Local\Temp\ipykernel_4288\1949036155.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  segmento_df[column] = segmento_df[column].apply(lambda x: extract_values_of_dictionaries(x, name))
C:\Users\sramr\AppData\Local\Temp\ipykernel_4288\1949036155.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  segmento_df[column] = segmento_df[column].apply(lambda x: extract_values_of_dictionaries(x, name))
C:\Users\sramr\AppData\Local\Temp\ipykernel_4288\1949036155.py:23: SettingWithCopyWarnin

In [113]:
df['genres']

0         ["Animation", "Comedy", "Family"]
1        ["Adventure", "Fantasy", "Family"]
2                     ["Romance", "Comedy"]
3            ["Comedy", "Drama", "Romance"]
4                                ["Comedy"]
                        ...                
45424                   ["Drama", "Family"]
45425                             ["Drama"]
45426       ["Action", "Drama", "Thriller"]
45427                                    []
45428                                    []
Name: genres, Length: 45429, dtype: object

In [114]:
df = convert_column_json(df, 'production_countries')

C:\Users\sramr\AppData\Local\Temp\ipykernel_4288\1949036155.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  segmento_df[column] = segmento_df[column].apply(lambda x: extract_values_of_dictionaries(x, name))
C:\Users\sramr\AppData\Local\Temp\ipykernel_4288\1949036155.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  segmento_df[column] = segmento_df[column].apply(lambda x: extract_values_of_dictionaries(x, name))
C:\Users\sramr\AppData\Local\Temp\ipykernel_4288\1949036155.py:23: SettingWithCopyWarnin

In [115]:
df = convert_column_json(df, 'spoken_languages')

C:\Users\sramr\AppData\Local\Temp\ipykernel_4288\1949036155.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  segmento_df[column] = segmento_df[column].apply(lambda x: extract_values_of_dictionaries(x, name))
C:\Users\sramr\AppData\Local\Temp\ipykernel_4288\1949036155.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  segmento_df[column] = segmento_df[column].apply(lambda x: extract_values_of_dictionaries(x, name))
C:\Users\sramr\AppData\Local\Temp\ipykernel_4288\1949036155.py:23: SettingWithCopyWarnin

In [116]:
df = convert_column_json(df, 'cast')

C:\Users\sramr\AppData\Local\Temp\ipykernel_4288\1949036155.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  segmento_df[column] = segmento_df[column].apply(lambda x: extract_values_of_dictionaries(x, name))
C:\Users\sramr\AppData\Local\Temp\ipykernel_4288\1949036155.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  segmento_df[column] = segmento_df[column].apply(lambda x: extract_values_of_dictionaries(x, name))
C:\Users\sramr\AppData\Local\Temp\ipykernel_4288\1949036155.py:23: SettingWithCopyWarnin

In [117]:
def convert_column_crew(df: pd.DataFrame, num_segmentos=507):
    '''
    Esta función transforma la columna 'crew' en un formato más simple
    Requiere el dataframe
    Retorna el dataframe con la columna modificada
    '''
    # Calcula el tamaño de cada segmento
    tam_segmento = len(df) // num_segmentos

    for i in range(num_segmentos):
        # Calcula los índices inicial y final para el segmento actual
        start_idx = i * tam_segmento
        end_idx = (i + 1) * tam_segmento

        # Si es el último segmento, ajusta el índice final al tamaño del DataFrame
        if i == num_segmentos - 1:
            end_idx = len(df)

        # Obtiene el segmento actual del DataFrame
        segmento_df = df.iloc[start_idx:end_idx]

        # Aplica la transformación a la columna 'crew' usando el método apply
        segmento_df['crew'] = segmento_df['crew'].apply(extract_director)

        # Hace que los cambios se vean reflejados en el dataframe original
        df.iloc[start_idx:end_idx] = segmento_df

    return df

def extract_director(value):
    # Transforma la fila en la columna 'crew' en una cadena de texto
    fila = str(value)
    # Utiliza expresión regular para encontrar los substrings entre {} (los diccionarios en la fila)
    diccionarios_en_la_fila = re.findall(r'{.*?}', fila)

    # Una lista de los directores asociados al film actual en el loop
    director_row = []

    for diccionario in diccionarios_en_la_fila:
        match = re.search(r"name': '([^']*)", diccionario)

        if match:
            director_h = match.group(1)

            match = re.search(r"job': '([^']*)", diccionario)

            if match:
                job = match.group(1)

                if job == 'Director' and director_h:
                    director_row.append(director_h)

    # Devuelve la lista de directores como string
    return json.dumps(director_row)

In [118]:
df = convert_column_crew(df)

C:\Users\sramr\AppData\Local\Temp\ipykernel_4288\934334754.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  segmento_df['crew'] = segmento_df['crew'].apply(extract_director)
C:\Users\sramr\AppData\Local\Temp\ipykernel_4288\934334754.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  segmento_df['crew'] = segmento_df['crew'].apply(extract_director)
C:\Users\sramr\AppData\Local\Temp\ipykernel_4288\934334754.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

- La decision es no considerar a las colecciones en el ml debido a que hay muchos datos faltantes. 

- **Transformacion 2:**
        - Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.

In [119]:
tupla1 = df['revenue'].isnull().sum(), df['budget'].isnull().sum()
print('(nulos_revenue, nulos_budget) = '+ str(tupla1))

(nulos_revenue, nulos_budget) = (0, 0)


In [120]:
# No hay valores nulos, pero de todas formas creamos una funcion 

def fillna_revenue_and_budget(df: pd.DataFrame):
    
    # Rellena los valores faltantes en la columna 'revenue' con 0
    df['revenue'].fillna(value=0, inplace=True)
    
    # Rellena los valores faltantes en la columna 'budget' con 0
    df['budget'].fillna(value=0, inplace=True)
    
    # Devuelve el DataFrame modificado
    return df


df = fillna_revenue_and_budget(df)

In [121]:
tupla2 = df['revenue'].isnull().sum(), df['budget'].isnull().sum()
print('(nulos_revenue, nulos_budget) = '+ str(tupla2))

(nulos_revenue, nulos_budget) = (0, 0)


- **Transformacion 3:** 
        - Los valores nulos del campo release date deben eliminarse.

In [122]:
tupla3 = df['release_date'].isnull().sum(), (df['release_date'] == '').sum()
print('(nulos_release_date, ''_release_date) = ' + str(tupla3))

(nulos_release_date, _release_date) = (84, 0)


In [123]:
def dropna_release_date(df: pd.DataFrame):
    
    # Convierte la columna 'release_date' al formato datetime, permitiendo errores y valores no válidos
    df['release_date'] = pd.to_datetime(df['release_date'], format='%Y-%m-%d', errors='coerce')
    
    # Elimina las filas que tienen valores faltantes en la columna 'release_date'
    df.dropna(subset=['release_date'], inplace=True)
    
    # Devuelve el DataFrame modificado
    return df


df = dropna_release_date(df)

In [124]:
tupla4 = df['release_date'].isnull().sum(), (df['release_date'] == '').sum()
print('(nulos_release_date, ''_release_date) = ' + str(tupla4))

(nulos_release_date, _release_date) = (0, 0)


- **Transformacion 4**:
        - De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

In [125]:
if  'release_year' in df.columns: 
    mensaje = "La columna 'release_year' está presente en el DataFrame." 
else:
    mensaje = "La columna 'release_year' no está presente en el DataFrame."
print(mensaje)

La columna 'release_year' no está presente en el DataFrame.


In [126]:
def add_release_year_column(df: pd.DataFrame):
    if 'release_year' in df.columns:
        df = df.drop('release_year', axis=1)  # Eliminar la columna 'release_year' si ya existe
        
    # Convierte la columna 'release_date' al formato datetime, permitiendo errores y valores no válidos
    df['release_date'] = pd.to_datetime(df['release_date'], format='%Y-%m-%d', errors='coerce')
    
    # Obtiene el índice de la columna 'release_date'
    release_date_idx = df.columns.get_loc('release_date')
    
    # Inserta una nueva columna 'release_year' después de la columna 'release_date'
    df.insert(release_date_idx + 1, 'release_year', df['release_date'].dt.strftime('%Y'))
    
    # Devuelve el DataFrame modificado
    return df


df = add_release_year_column(df)

In [127]:
if  'release_year' in df.columns: 
    mensaje = "La columna 'release_year' está presente en el DataFrame." 
else:
    mensaje = "La columna 'release_year' no está presente en el DataFrame."
print(mensaje)

La columna 'release_year' está presente en el DataFrame.


- **Transformacion 5**:
      - Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

In [128]:
if  'return' in df.columns: 
    mensaje = "La columna 'return' está presente en el DataFrame." 
else:
    mensaje = "La columna 'return' no está presente en el DataFrame."
print(mensaje)

La columna 'return' no está presente en el DataFrame.


In [129]:
def add_return_column(df: pd.DataFrame):
    
    if 'return' in df.columns:
        df = df.drop('return', axis=1)  # Eliminar la columna 'inversion' si ya existe
        
    # Convertir las columnas revenue y budget en numéricas
    df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')
    df['budget'] = pd.to_numeric(df['budget'], errors='coerce')
    
    # Calcular el retorno de inversión
    df['return'] = df.apply(lambda row: row['revenue'] / row['budget'] if (pd.notnull(row['revenue']) and pd.notnull(row['budget']) and row['budget']!=0) else 0, axis=1)
    
    return df  


df = add_return_column(df)

In [130]:
if  'return' in df.columns: 
    mensaje = "La columna 'return' está presente en el DataFrame." 
else:
    mensaje = "La columna 'return' no está presente en el DataFrame."
print(mensaje)

La columna 'return' está presente en el DataFrame.


- **Transformacion 6**:
        - Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage

In [131]:
def drop_useless_columns(df: pd.DataFrame):
    
    # En realidad voy a eliminar todas las columnas solicitadas a excepcion de 'adult' ya que la considero util en el ml
    columns_to_drop = ['video', 'imdb_id', 'original_title', 'poster_path', 'homepage']

    # Filtra las columnas existentes en el DataFrame
    columns_to_drop = [column for column in columns_to_drop if column in df.columns]
    
    # Verifica si hay columnas para eliminar
    if columns_to_drop:
        
        # Elimina las columnas especificadas del DataFrame
        df.drop(columns_to_drop, axis=1, inplace=True)
        
    # Devuelve el DataFrame modificado
    return df


df = drop_useless_columns(df)

# Load

In [132]:
# Guardamos los cambios en el mismo csv
df.to_csv("../../data/data_procesada/data_procesada.csv", encoding='utf-8-sig', index=False, sep=',') 